In [1]:
# pip install numpy==1.19.2 

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.layers as layers
# from data_generator import NpyDataGenerator
from hyperopt import hp, tpe, Trials, fmin
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from Logit_polynomial_V12 import SVM as opt_svm
from tqdm import tqdm
from collections import Counter
import numpy as np
from tensorflow.keras.optimizers import RMSprop, SGD, Adam,Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
# import einops
from data_generator_v2 import NpyDataGenerator

In [3]:
# os.listdir()

In [4]:
train_path = "/media/kashraf/TOSHIBA EXT/Dissertation/stage2/wavelet/Mean_power_wt/Beta/train/"
test_path = "/media/kashraf/TOSHIBA EXT/Dissertation/stage2/wavelet/Mean_power_wt/Beta/test/"
train_gen = NpyDataGenerator(train_path,batch_size=32)
validation_gen = NpyDataGenerator(test_path,batch_size=1,shuffle=False)
print("Training samples: ",train_gen.num_samples)
print("Test samples: ",validation_gen.num_samples)

Training samples:  42000
Test samples:  18000


In [5]:
# import tensorflow as tf
# from tensorflow.keras import layers, models

# # Define the model
# model = models.Sequential()

# # First LSTM layer with 128 units
# model.add(layers.LSTM(64, input_shape=(176, 64), return_sequences=True))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.3))

# # Second LSTM layer with 64 units
# model.add(layers.LSTM(64, return_sequences=True))
# model.add(layers.BatchNormalization())
# # model.add(layers.Dropout(0.3))

# # Second LSTM layer with 64 units
# model.add(layers.LSTM(128, return_sequences=True))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.3))

# # Second LSTM layer with 64 units
# model.add(layers.LSTM(128, return_sequences=True))
# model.add(layers.BatchNormalization())
# # model.add(layers.Dropout(0.3))

# # model.add(layers.LSTM(512, return_sequences=False))
# # model.add(layers.BatchNormalization())
# # # model.add(layers.Dropout(0.3))

# # # Dense layer with 32 units
# # model.add(layers.Dense(256, activation='relu'))
# # model.add(layers.BatchNormalization())
# # model.add(layers.Dropout(0.3))
# # Dense layer with 32 units
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.BatchNormalization())
# # model.add(layers.Dropout(0.3))
# model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.3))

# # Output layer for 4-class classification
# model.add(layers.Dense(4, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Summary of the model
# model.summary()


In [6]:
             
# checkpoint = ModelCheckpoint("/media/kashraf/Elements/Dissertation/modelling/General/weights/Transformer_mean_wt.h5",
#                              monitor="val_loss",
#                              mode="min",
#                              save_best_only = True,
#                              verbose=1)

# earlystop = EarlyStopping(monitor = 'val_loss', 
#                           min_delta = 0, 
#                           patience = 10,
#                           verbose = 1,
#                           restore_best_weights = True)

# reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
#                               factor = 0.01,
#                               patience = 3,
#                               verbose = 20,
#                               min_delta = 0.000001)

# # we put our call backs into a callback list
# callbacks = [ checkpoint,reduce_lr,earlystop]

# # We use a very small learning rate 
# model.compile(loss = 'categorical_crossentropy',
#               optimizer = Adam(),
#               metrics = ['accuracy'])


# epochs = 100

# history= model.fit(   
#     train_gen,
#     epochs = epochs,
#     batch_size=32,
#     callbacks =callbacks,
#     validation_data = validation_gen)

## Hyperopt

In [7]:
# space = {
#     'num_layers': hp.choice('num_layers', [1, 2, 3,4,5]),
#     'units': hp.choice('units', [16,32,64, 128, 256]),
#     'dropout': hp.uniform('dropout', 0, 0.5),
#     'learning_rate': hp.loguniform('learning_rate', -5, -2),
#     'epochs': hp.choice('epochs', [50,100])
# }

# def objective(params):
#     # Create LSTM model with hyperparameters
#     model = Sequential()
#     for i in range(params['num_layers']):
#         if i == 0:
#             model.add(LSTM(params['units'], input_shape=(176, 64), dropout=params['dropout'], return_sequences=True))
#         elif i == params['num_layers']-1:
#             model.add(LSTM(params['units'], dropout=params['dropout']))
#         else:
#             model.add(LSTM(params['units'], dropout=params['dropout'], return_sequences=True))
#     model.add(Flatten())
#     model.add(Dense(num_classes, activation='softmax'))

#     # Compile model with hyperparameters
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#     # Train model with hyperparameters
#     model.fit(train_gen, epochs=50, batch_size=32, validation_data=validation_gen, verbose=2)

#     # Evaluate model on validation set
#     loss, accuracy = model.evaluate(validation_gen,verbose=0)

#     # Return validation accuracy as the objective value
#     return {'loss': -accuracy, 'status': 'ok'}

# trials = Trials()
# num_classes= 4
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials,verbose=True)

# # Print best hyperparameters
# print('Best hyperparameters:', best)



In [16]:
import pickle
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
num_classes= 4
# Define hyperparameter search space
space = {
    'num_layers': hp.choice('num_layers', [1, 2, 3, 4, 5]),
    'units': hp.choice('units', [16, 32, 64, 128, 256]),
    'dropout': hp.uniform('dropout', 0, 0.5),
    'recurrent_dropout': hp.uniform('recurrent_dropout', 0, 0.5),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'activation': hp.choice('activation', ['tanh', 'relu', 'sigmoid']),
    'optimizer': hp.choice('optimizer', ['adam', 'rmsprop', 'sgd']),
    'l2_reg': hp.loguniform('l2_reg', -6, -3),
    'clipvalue': hp.uniform('clipvalue', 1, 5),
    'bidirectional': hp.choice('bidirectional', [True, False])
}

# Initialize Trials object
trials = Trials()

# Function to save the best hyperparameters to a pickle file
def save_best_params(best_params):
    with open('best_params.pkl', 'wb') as f:
        pickle.dump(best_params, f)

# Function to load the best hyperparameters from the pickle file
def load_best_params():
    try:
        with open('best_params.pkl', 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

# Objective function
def objective(params):
    model = Sequential()

    for i in range(params['num_layers']):
        if i == 0:  # First layer with input shape
            if params['bidirectional']:
                model.add(Bidirectional(LSTM(params['units'], activation='tanh', 
                                             recurrent_activation='sigmoid',
                                             dropout=params['dropout'], recurrent_dropout=0, 
                                             return_sequences=True, kernel_regularizer=l2(params['l2_reg'])),
                                             input_shape=(176, 64)))
            else:
                model.add(LSTM(params['units'], activation='tanh', recurrent_activation='sigmoid',
                               dropout=params['dropout'], recurrent_dropout=0, 
                               return_sequences=True, kernel_regularizer=l2(params['l2_reg']),
                               input_shape=(176, 64)))
        elif i == params['num_layers'] - 1:  # Last LSTM layer
            if params['bidirectional']:
                model.add(Bidirectional(LSTM(params['units'], activation='tanh', 
                                             recurrent_activation='sigmoid',
                                             dropout=params['dropout'], recurrent_dropout=0, 
                                             kernel_regularizer=l2(params['l2_reg']))))
            else:
                model.add(LSTM(params['units'], activation='tanh', recurrent_activation='sigmoid',
                               dropout=params['dropout'], recurrent_dropout=0, 
                               kernel_regularizer=l2(params['l2_reg'])))
        else:  # Middle LSTM layers
            if params['bidirectional']:
                model.add(Bidirectional(LSTM(params['units'], activation='tanh', 
                                             recurrent_activation='sigmoid',
                                             dropout=params['dropout'], recurrent_dropout=0, 
                                             return_sequences=True, kernel_regularizer=l2(params['l2_reg']))))
            else:
                model.add(LSTM(params['units'], activation='tanh', recurrent_activation='sigmoid',
                               dropout=params['dropout'], recurrent_dropout=0, 
                               return_sequences=True, kernel_regularizer=l2(params['l2_reg'])))
    
    # Add output layers
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    # Optimizer and other code remain the same

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=params["optimizer"], metrics=['accuracy'])

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)

    # Train the model
    model.fit(train_gen, epochs=50, batch_size=32, validation_data=validation_gen, 
              callbacks=[early_stopping], verbose=1)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(validation_gen, verbose=1)

    # Return the negative accuracy as the objective value (for minimization)
    return {'loss': -accuracy, 'status': STATUS_OK}


# Callback to print and save the best parameters after each trial
def trial_callback(trial):
    best_trial = trials.best_trial
    print(f"Trial {len(trials.trials)} completed with result: {trial['result']}")
    print(f"Best parameters so far: {best_trial['result']}")
    save_best_params(best_trial['result'])

# Load the best hyperparameters from previous session (if any)
best_params = load_best_params()
if best_params:
    print(f"Loaded best parameters from previous session: {best_params}")

# Run the hyperparameter optimization
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=25, trials=trials, verbose=True, 
            catch_eval_exceptions=True)

# Save the final best hyperparameters to the pickle file
save_best_params(best)
print('Best hyperparameters:', best)


Epoch 1/50                                                                                                                                                                                                 
   1/1313 [..............................]                                                                                                                                                                 
 - ETA: 58:56 - loss: 2.8490 - accuracy: 0.3125                                                                                                                                                            
                                                                                                                                                                                                           
   3/1313 [..............................]                                                                                                                                              

  15/1313 [..............................]                                                                                                                                                                 
 - ETA: 41:13 - loss: 2.4467 - accuracy: 0.2695                                                                                                                                                            
                                                                                                                                                                                                           
  16/1313 [..............................]                                                                                                                                                                 
 - ETA: 41:20 - loss: 2.4348 - accuracy: 0.2696                                                                                                                                         

 - ETA: 40:51 - loss: 2.3321 - accuracy: 0.2779                                                                                                                                                            
                                                                                                                                                                                                           
  30/1313 [..............................]                                                                                                                                                                 
 - ETA: 40:57 - loss: 2.3263 - accuracy: 0.2781                                                                                                                                                            
                                                                                                                                                                                        

                                                                                                                                                                                                           
  45/1313 [>.............................]                                                                                                                                                                 
 - ETA: 38:44 - loss: 2.2537 - accuracy: 0.2774                                                                                                                                                            
                                                                                                                                                                                                           
  46/1313 [>.............................]                                                                                                                                              

  60/1313 [>.............................]                                                                                                                                                                 
 - ETA: 37:32 - loss: 2.1987 - accuracy: 0.2752                                                                                                                                                            
                                                                                                                                                                                                           
  61/1313 [>.............................]                                                                                                                                                                 
 - ETA: 37:25 - loss: 2.1953 - accuracy: 0.2750                                                                                                                                         

...

                                                                                                                                                                                                           
 607/1313 [============>.................]                                                                                                                                                                 
 - ETA: 17:47 - loss: 1.6778 - accuracy: 0.2876                                                                                                                                                            
                                                                                                                                                                                                           
 608/1313 [============>.................]                                                                                                                                              

 621/1313 [=============>................]                                                                                                                                                                 
 - ETA: 17:20 - loss: 1.6734 - accuracy: 0.2881                                                                                                                                                            
                                                                                                                                                                                                           
 622/1313 [=============>................]                                                                                                                                                                 
 - ETA: 17:18 - loss: 1.6731 - accuracy: 0.2882                                                                                                                                         

 - ETA: 16:50 - loss: 1.6689 - accuracy: 0.2887                                                                                                                                                            
                                                                                                                                                                                                           
 637/1313 [=============>................]                                                                                                                                                                 
 - ETA: 16:48 - loss: 1.6686 - accuracy: 0.2888                                                                                                                                                            
                                                                                                                                                                                        

                                                                                                                                                                                                           
 652/1313 [=============>................]                                                                                                                                                                 
 - ETA: 16:20 - loss: 1.6642 - accuracy: 0.2893                                                                                                                                                            
                                                                                                                                                                                                           
 654/1313 [=============>................]                                                                                                                                              

KeyboardInterrupt: 

In [ ]:
trials = Trials()
num_classes= 4
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials,verbose=True)

# Print best hyperparameters
print('Best hyperparameters:', best)



In [ ]:
# Train final model using best hyperparameters
best_units = space['units'][best['units']]
best_dropout = best['dropout']
best_batch_size = space['batch_size'][best['batch_size']]
best_learning_rate = best['learning_rate']
best_epochs = space['epochs'][best['epochs']]
best_model = Sequential()
best_model.add(LSTM(best_units, input_shape=(176, 64), dropout=best_dropout))
best_model.add(Dense(num_classes, activation='softmax'))
best_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
best_model.fit(x_train, y_train, epochs=int(best_epochs), batch_size=int(best_batch_size),validation_data= (x_test,y_test),verbose=1)